# Arize Phoenix


## What is Arize Phoenix?

Phoenix is a platform designed to enable observability across every layer of an LLM-based system. It is an open-source observability tool for experimentation, evaluation, and troubleshooting of AI and LLM applications. It allows AI engineers and data scientists to quickly visualize their data, evaluate performance, track down issues, and export data to improve. It supports teams to build, optimize, and maintain high-quality applications and agents efficiently.

## Tracing

Tracing is a tool for understanding how your LLM application behaves. Phoenix's open-source library offers comprehensive tracing capabilities that are not tied to any specific LLM vendor or framework. :

- `Tavily Search`: A search engine designed for AI agents, combining search and scraping capabilities.
- `Tavily Extract`: Web Scraping for up to 20 URLs in a single API call.
- `Tavily Crawl`: Extensive crawl of multiple domains.

## Why Tavily is different

Tavily dynamically searches the web, reviews multiple sources, and extracts the most relevant and concise ready-to-use information optimized for AI applications. It focuses on delivering high-quality, customizable search results. Developers control search depth, domain targeting, and content extraction. LLM-generated answers are optional, making Tavily a flexible, search-first solution adaptable to different use cases.

<img justify='center' src="https://arize.com/docs/phoenix/~gitbook/image?url=https%3A%2F%2Fstorage.googleapis.com%2Farize-assets%2Fphoenix%2Fassets%2Fimages%2Fspan_kinds.png&width=768&dpr=2&quality=100&sign=f3cfa40a&sv=2" alt="Phoenix Tracing Spans" width="700">


## Phoenix Quickstart

In [ ]:
!pip install -q arize-phoenix-otel
!pip install -q openinference-instrumentation-google-genai
!pip install -q openinference-instrumentation-langchain
!pip install -q langchain-google-genai
!pip install -q openinference-instrumentation

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path
from phoenix.otel import register
from google import genai

In [2]:
## Load the Environment variables

load_dotenv(Path('../../.env'))
os.environ["PHOENIX_API_KEY"] = os.getenv("PHOENIX_API_KEY")
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = os.getenv("PHOENIX_COLLECTOR_ENDPOINT")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
# os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")

In [3]:
# configure the Phoenix tracer
tracer_provider = register(
  project_name="GOOGLE-llm-app", # If not defined, will set to Default automatically'
  auto_instrument=True,
)
tracer = tracer_provider.get_tracer(__name__)

/Users/user1/AI/AI_Tutorial/GenAI_Tutorial/.venv/lib/python3.12/site-packages/phoenix/otel/otel.py:333: UserWarning: Could not infer collector endpoint protocol, defaulting to HTTP.
  warnings.warn("Could not infer collector endpoint protocol, defaulting to HTTP.")


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: GOOGLE-llm-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/s/comradedaniel/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



Arize suppports the tracing of popular LLM frameworks. You can view them here: <https://github.com/Arize-ai/openinference?tab=readme-ov-file#instrumentation>.

In [4]:
client = genai.Client()
chat = client.chats.create(model="gemini-2.0-flash-001")
response = chat.send_message("What is the capital of the common provinces in South Africa?")

## LangChain

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

prompt = ChatPromptTemplate.from_template("{x} {y} {z}?").partial(x="why is", z="blue")
chain = prompt | ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
chain.invoke(dict(y="sky"))

AIMessage(content="The sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's a breakdown:\n\n*   **Sunlight is white:** Sunlight is actually a mixture of all the colors of the rainbow.\n\n*   **Earth's atmosphere:** The Earth is surrounded by an atmosphere containing various gases and particles (mostly nitrogen and oxygen molecules).\n\n*   **Scattering:** When sunlight enters the atmosphere, it collides with these air molecules. This collision causes the light to scatter in different directions.\n\n*   **Rayleigh scattering:** Rayleigh scattering is a specific type of scattering that is most effective when the size of the particles (air molecules) is much smaller than the wavelength of the light.\n\n*   **Wavelength and scattering:** Blue and violet light have shorter wavelengths compared to other colors like red and orange. Rayleigh scattering is much more effective at scattering shorter wavelengths. The amount of scattering is inversely proportional to the four

## Sessions

In [ ]:
import uuid

import google.generativeai as genai
from openinference.instrumentation import using_session
from openinference.semconv.trace import SpanAttributes
from opentelemetry import trace

client = genai.GenerativeModel("gemini-2.0-flash-001")
session_id = str(uuid.uuid4())

tracer = trace.get_tracer(__name__)

@tracer.start_as_current_span(name="agent", attributes={SpanAttributes.OPENINFERENCE_SPAN_KIND: "agent"})
def assistant(
  messages: list[dict],
  session_id: str = str,
):
  current_span = trace.get_current_span()
  current_span.set_attribute(SpanAttributes.SESSION_ID, session_id)
  current_span.set_attribute(SpanAttributes.INPUT_VALUE, messages[-1].get('content'))

  # Propagate the session_id down to spans crated by the OpenAI instrumentation
  # This is not strictly necessary, but it helps to correlate the spans to the same session
  with using_session(session_id):
   response = client.chat.completions.create(
       model="gpt-3.5-turbo",
       messages=[{"role": "system", "content": "You are a helpful assistant."}] + messages,
   ).choices[0].message

  current_span.set_attribute(SpanAttributes.OUTPUT_VALUE, response.content)
  return response

messages = [
  {"role": "user", "content": "hi! im bob"}
]
response = assistant(
  messages,
  session_id=session_id,
)
messages = messages + [
  response,
  {"role": "user", "content": "what's my name?"}
]
response = assistant(
  messages,
  session_id=session_id,
)